In [189]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import re
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
plt.style.use('ggplot')

In [190]:
ipl_data = pd.read_csv('IPL.csv')
match_dates = pd.read_csv('match_dates.csv')

In [191]:
ipl_data.head(10)

,team_1,team_2,team_1_score,team_2_score,venue,winner
0,Mumbai Indians,Chennai Super Kings,149/8,148/7,Hyderabad,Mumbai Indians won by 1 run
1,Chennai Super Kings,Delhi Capitals,151/4,147/9,Visakhapatnam,Chennai Super Kings won by 6 wickets
2,Delhi Capitals,Sunrisers Hyderabad,165/8,162/8,Visakhapatnam,Delhi Capitals won by 2 wickets
3,Mumbai Indians,Chennai Super Kings,132/4,131/4,Chennai,Mumbai Indians won by 6 wickets
4,Mumbai Indians,Kolkata Knight Riders,134/1,133/7,Mumbai,Mumbai Indians won by 9 wickets
5,Kings XI Punjab,Chennai Super Kings,173/4,170/5,Mohali,Kings XI Punjab won by 6 wickets
6,Royal Challengers Bangalore,Sunrisers Hyderabad,178/6,175/7,Bengaluru,Royal Challengers Bangalore won by 4 wickets
7,Delhi Capitals,Rajasthan Royals,121/5,115/9,Delhi,Delhi Capitals won by 5 wickets
8,Kings XI Punjab,Kolkata Knight Riders,183/6,185/3,Mohali,Kolkata Knight Riders won by 7 wickets
9,Mumbai Indians,Sunrisers Hyderabad,162/5,162/6,Mumbai,Match Tied (Mumbai Indians won the Super Over)


In [192]:
match_dates.head(10)

,match_date,match_day
0,"['12th', 'May']",Sunday
1,"['10th', 'May']",Friday
2,"['8th', 'May']",Wednesday
3,"['7th', 'May']",Tuesday
4,"['5th', 'May']",Sunday
5,"['5th', 'May']",Sunday
6,"['4th', 'May']",Saturday
7,"['4th', 'May']",Saturday
8,"['3rd', 'May']",Friday
9,"['2nd', 'May']",Thursday


In [193]:
main_df = pd.concat([ipl_data, match_dates], axis=1)

In [194]:
main_df.head(10)

,team_1,team_2,team_1_score,team_2_score,venue,winner,match_date,match_day
0,Mumbai Indians,Chennai Super Kings,149/8,148/7,Hyderabad,Mumbai Indians won by 1 run,"['12th', 'May']",Sunday
1,Chennai Super Kings,Delhi Capitals,151/4,147/9,Visakhapatnam,Chennai Super Kings won by 6 wickets,"['10th', 'May']",Friday
2,Delhi Capitals,Sunrisers Hyderabad,165/8,162/8,Visakhapatnam,Delhi Capitals won by 2 wickets,"['8th', 'May']",Wednesday
3,Mumbai Indians,Chennai Super Kings,132/4,131/4,Chennai,Mumbai Indians won by 6 wickets,"['7th', 'May']",Tuesday
4,Mumbai Indians,Kolkata Knight Riders,134/1,133/7,Mumbai,Mumbai Indians won by 9 wickets,"['5th', 'May']",Sunday
5,Kings XI Punjab,Chennai Super Kings,173/4,170/5,Mohali,Kings XI Punjab won by 6 wickets,"['5th', 'May']",Sunday
6,Royal Challengers Bangalore,Sunrisers Hyderabad,178/6,175/7,Bengaluru,Royal Challengers Bangalore won by 4 wickets,"['4th', 'May']",Saturday
7,Delhi Capitals,Rajasthan Royals,121/5,115/9,Delhi,Delhi Capitals won by 5 wickets,"['4th', 'May']",Saturday
8,Kings XI Punjab,Kolkata Knight Riders,183/6,185/3,Mohali,Kolkata Knight Riders won by 7 wickets,"['3rd', 'May']",Friday
9,Mumbai Indians,Sunrisers Hyderabad,162/5,162/6,Mumbai,Match Tied (Mumbai Indians won the Super Over),"['2nd', 'May']",Thursday


In [195]:
"""Extracting Winner from winner columns"""
for i in range(60):
    if 'Match tied' in main_df['winner'][i]: #corner case for if match tied
        main_df['winner'][i] = main_df['winner'][9].split('won')[0].split('(')[0]
        
    elif 'Match Tied' in main_df['winner'][i]: #corner case for if match tied
        main_df['winner'][i] = main_df['winner'][9].split('won')[0].split('(')[0]
        
    elif 'No Result' in main_df['winner'][i]: #corner case if No Result
        main_df['winner'][i] = main_df['winner'][i]
        
    main_df['winner'][i] = main_df['winner'][i].split('won')[0] #Regular Winner

In [196]:
#mappings for team and winner
team_mapping = {'Mumbai Indians': 'MI','Chennai Super Kings': 'CSK', 'Sunrisers Hyderabad': 'SRH',
                'Kolkata Knight Riders': 'KKR',  'Rajasthan Royals': 'RR', 'Kings XI Punjab': 'KXIP',
                'Delhi Capitals':'DC', 'Royal Challengers Bangalore': 'RCB', 'No Result': 'NR'
               }

winner_mapping = {'Mumbai Indians ': 'MI', 'Chennai Super Kings ': 'CSK',
                 'Delhi Capitals ': 'DC', 'Kings XI Punjab ': 'KXIP',
                 'Royal Challengers Bangalore ': 'RCB', 'Kolkata Knight Riders ': 'KKR',
                 'Sunrisers Hyderabad ': 'SRH', 'Rajasthan Royals ': 'RR', 
                  'Match Tied ': 'Match Tied', 'No Result' : 'No Result'}

In [197]:
for i in main_df[['team_1', 'team_2']]:
    main_df[i] = main_df[i].map(team_mapping)



main_df['winner'] = main_df['winner'].map(winner_mapping)

In [198]:
#Extract Match date
for i in range(len(main_df['match_date'])):
    split_date = main_df['match_date'][i].split('[')[1]
    new_date = re.findall('(?:\d{1,2}[-/th|st|nd|rd\s]*)?(?:March|April|May)?', split_date)
    main_df['match_date'][i] = new_date[1] + ' ' + new_date[6]

In [216]:
def get_score(series):
    """Remove wickets from score and return score only"""
    for i in range(len(series)):
        if len(series[i]) > 3:
            series[i] = series[i][:-2]
        else:
            pass
    return series

In [221]:
main_df['team_1_score'] = get_score(main_df['team_1_score'])
main_df['team_2_score'] = get_score(main_df['team_2_score'])

In [225]:
main_df.head(10)

,team_1,team_2,team_1_score,team_2_score,venue,winner,match_date,match_day
0,MI,CSK,149,148,Hyderabad,MI,12th May,Sunday
1,CSK,DC,151,147,Visakhapatnam,CSK,10th May,Friday
2,DC,SRH,165,162,Visakhapatnam,DC,8th May,Wednesday
3,MI,CSK,132,131,Chennai,MI,7th May,Tuesday
4,MI,KKR,134,133,Mumbai,MI,5th May,Sunday
5,KXIP,CSK,173,170,Mohali,KXIP,5th May,Sunday
6,RCB,SRH,178,175,Bengaluru,RCB,4th May,Saturday
7,DC,RR,121,115,Delhi,DC,4th May,Saturday
8,KXIP,KKR,183,185,Mohali,KKR,3rd May,Friday
9,MI,SRH,162,162,Mumbai,Match Tied,2nd May,Thursday


In [227]:
main_df.to_csv('preprocessed_IPL.csv', index=False)